# PROJEKTS: Knowledge Hub



Situācija
Tu esi tikko pieņemts darbā par MI Inženieri. Uzņēmuma vadītājs tev iedevis mapi UNSORTED_DATA, kurā ir haoss – sapulču ieraksti, tāfeles fotogrāfijas un līgumu melnraksti.

Nevienam nav laika to visu lasīt vai klausīties.

## Tavs Uzdevums:

Uzbūvēt Multimodālu RAG Sistēmu, kas "apēd" šos failus un ļauj direktoram čatā uzdot jautājumus (piemēram: "Cik mēs esam parādā?" vai "Ko nolēma par mārketingu?"), saņemot precīzas atbildes ar atsaucēm.

## Vides sagatavošana

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Instalējam nepieciešamās bibliotēkas
!pip install openai langchain langchain-community langchain-openai chromadb pypdf openai-whisper tiktoken
!apt-get install ffmpeg -y  # Nepieciešams priekš Whisper audio apstrādes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 75.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 M

In [ ]:
import os
import getpass
from pathlib import Path
from google.colab import drive

# 1. Pievienojam Google Drive (ja vēl nav pievienots)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Definējam darba mapi (ceļš no tava ekrānšāviņa)
BASE_PATH = Path("/content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub")

# 3. Ievadi OpenAI API atslēgu
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ievadi OpenAI API Key: ")

# 4. Pārbaudām failus
required_files = ["project.pdf", "meeting.mp3", "whiteboard.png"] # Pamanīju, ka tev ir .png
missing_files = [f for f in required_files if not (BASE_PATH / f).exists()]

if missing_files:
    print(f"❌ TRŪKST FAILU mapē {BASE_PATH}: {missing_files}")
    print("Pārbaudi failu nosaukumus!")
else:
    print(f"✅ Visi faili atrasti mapē: {BASE_PATH}")
    # Iestatām pilnos ceļus ērtākai lietošanai
    pdf_path = str(BASE_PATH / "project.pdf")
    audio_path = str(BASE_PATH / "meeting.mp3")
    image_path = str(BASE_PATH / "whiteboard.png")

✅ Visi faili atrasti mapē: /content/drive/MyDrive/Colab Notebooks/BDA_M1/Lab26_KnowledgeHub


## Izstrādes soļi (Ingestion & Vector Store)
Šis kods "apēd" failus no Google Drive mapes.
* Whisper lasa audio no Drive ceļa.
* GPT-4o lasa attēlu no Drive ceļa.
* PyPDFLoader lasa PDF no Drive ceļa.

In [8]:
import whisper
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from openai import OpenAI
import base64

# Iniciējam OpenAI klientu attēlu apstrādei
client = OpenAI()

print("⏳ Sāku datu apstrādi... (Tas var aizņemt pāris minūtes)")
all_documents = []

# --- A. DATU "APĒŠANA" (Ingestion) ---

# 1. AUDIO APSTRĀDE (Whisper)
print(f"   🎤 Apstrādāju audio: {os.path.basename(audio_path)}...")
whisper_model = whisper.load_model("base")
# Whisper ņem faila ceļu tieši no Drive
result = whisper_model.transcribe(audio_path)
audio_text = result["text"]
all_documents.append(Document(page_content=audio_text, metadata={"source": "meeting.mp3"}))


# 2. ATTĒLA APSTRĀDE (GPT-4o Vision)
print(f"   🖼️ Apstrādāju attēlu: {os.path.basename(image_path)}...")

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Kodējam attēlu no Drive
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Lūdzu, detalizēti apraksti šo attēlu. Ja redzi tabulu vai skaitļus (budžetu, izmaksas), pārraksti tos precīzi teksta formātā."},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ],
        }
    ],
    max_tokens=1000,
)
image_description = response.choices[0].message.content
all_documents.append(Document(page_content=image_description, metadata={"source": "whiteboard.png"}))


# 3. PDF APSTRĀDE
print(f"   📄 Apstrādāju PDF: {os.path.basename(pdf_path)}...")
loader = PyPDFLoader(pdf_path)
pdf_docs = loader.load()
# Pievienojam PDF lapas kopējam sarakstam
all_documents.extend(pdf_docs)


# --- B. ZINĀŠANU BĀZE (Vector Store) ---

print("   💾 Veidoju Vektoru Datubāzi (ChromaDB)...")

# Sadalām tekstu gabalos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(all_documents)

# Saglabājam ChromaDB (operatīvajā atmiņā)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small")
)

print(f"✅ Datu bāze gatava! Apstrādāti {len(splits)} informācijas gabali.")

⏳ Sāku datu apstrādi... (Tas var aizņemt pāris minūtes)
   🎤 Apstrādāju audio: meeting.mp3...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 170MiB/s]


   🖼️ Apstrādāju attēlu: whiteboard.png...
   📄 Apstrādāju PDF: project.pdf...


   💾 Veidoju Vektoru Datubāzi (ChromaDB)...
✅ Datu bāze gatava! Apstrādāti 40 informācijas gabali.


## Kods čata interfeisam (RAG funkcija)

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# --- RAG SISTĒMAS UZBŪVE ---

# 1. Definējam LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 2. Definējam Meklētāju
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Definējam Promptu
template = """Tu esi uzņēmuma stratēģiskais AI asistents.
Tavs uzdevums ir atbildēt uz jautājumiem, izmantojot TIKAI zemāk sniegto kontekstu.
Ja atbildes nav kontekstā, saki "Diemžēl failos šādas informācijas nav."
Vienmēr norādi avota failu iekavās (piem., [Avots: meeting.mp3]).

Konteksts:
{context}

Jautājums: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 4. Formatēšanas funkcija (lai redzētu avotus)
def format_docs(docs):
    return "\n\n".join([f"Saturs: {d.page_content}\n[Avots: {d.metadata.get('source', 'Nezināms')}]" for d in docs])

# 5. RAG ķēde
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- ČATA CILPA ---

print("🤖 Projekta Smadzenes ir gatavas. Jautā man jebko par failiem!")
print("Lai beigtu darbu, ieraksti: exit")

while True:
    # 1. Ievades lauks
    user_question = input("\nTavs Jautājums: ").strip()

    # 2. Izejas nosacījums
    if not user_question: continue
    if user_question.lower() in ["exit", "quit", "beigt", "-"]:
        print("👋 Čats pabeigts.")
        break

    # 3. RAG izpilde
    try:
        print("   🔍 Analizēju datus...")
        response = rag_chain.invoke(user_question)
        
        # 4. Rezultāts
        print(f"\nJautajums:\n{user_question}")
        print(f"\nAtbilde:\n{response}")
        print("-" * 50)
        
    except Exception as e:
        print(f"⚠️ Kļūda: {e}")

🤖 Projekta Smadzenes ir gatavas. Jautā man jebko par failiem!
Lai beigtu darbu, ieraksti: exit
   🔍 Analizēju datus...

Jautajums:
Kādi skaitļi redzami uz tāfeles/rēķinā?

Atbilde:
Diemžēl failos šādas informācijas nav.
--------------------------------------------------
👋 Čats pabeigts.
